In [1]:
# Betweenness Centrality (Centrality Measure)
# Difficulty Level : Expert
# Last Updated : 10 Feb, 2018
# In graph theory, betweenness centrality is a measure of centrality in a graph based on shortest paths. For every pair of vertices in a connected graph, there exists at least one shortest path between the vertices such that either the number of edges that the path passes through (for unweighted graphs) or the sum of the weights of the edges (for weighted graphs) is minimized. The betweenness centrality for each vertex is the number of these shortest paths that pass through the vertex.

# Betweenness centrality finds wide application in network theory: it represents the degree of which nodes stand between each other. For example, in a telecommunications network, a node with higher betweenness centrality would have more control over the network, because more information will pass through that node. Betweenness centrality was devised as a general measure of centrality: it applies to a wide range of problems in network theory, including problems related to social networks, biology, transport and scientific cooperation.

# Definition

# The betweenness centrality of a node {\displaystyle v} v is given by the expression:

#  g(v)=\sum _{{s\neq v\neq t}}{\frac  {\sigma _{{st}}(v)}{\sigma _{{st}}}}



# where \sigma_{st} is the total number of shortest paths from node s to node  t and  \sigma_{st}(v) is the number of those paths that pass through  v.

# Note that the betweenness centrality of a node scales with the number of pairs of nodes as implied by the summation indices. Therefore, the calculation may be rescaled by dividing through by the number of pairs of nodes not including  v, so that g\in [0,1]. The division is done by  (N-1)(N-2) for directed graphs and  (N-1)(N-2)/2 for undirected graphs, where  N is the number of nodes in the giant component. Note that this scales for the highest possible value, where one node is crossed by every single shortest path. This is often not the case, and a normalization can be performed without a loss of precision

# {\mbox{normal}}(g(v))={\frac  {g(v)-\min(g)}{\max(g)-\min(g)}}
# which results in:

#  \max(normal)=1
#  \min(normal)=0
# Note that this will always be a scaling from a smaller range into a larger range, so no precision is lost.

# Weighted Networks
# In a weighted network the links connecting the nodes are no longer treated as binary interactions, but are weighted in proportion to their capacity, influence, frequency, etc., which adds another dimension of heterogeneity within the network beyond the topological effects. A node’s strength in a weighted network is given by the sum of the weights of its adjacent edges.

#  s_{{i}}=\sum _{{j=1}}^{{N}}a_{{ij}}w_{{ij}}
# With a_{ij} and  w_{ij} being adjacency and weight matrices between nodes  i and  j, respectively. Analogous to the power law distribution of degree found in scale free networks, the strength of a given node follows a power law distribution as well.

#  s(k)\approx k^{\beta }
# A study of the average value  s(b) of the strength for vertices with betweenness  b shows that the functional behavior can be approximated by a scaling form

#  s(b)\approx b^{{\alpha }}

In [2]:
def betweenness_centrality(G, k=None, normalized=True, weight=None,
						endpoints=False, seed=None):
	r"""Compute the shortest-path betweenness centrality for nodes.

	Betweenness centrality of a node $v$ is the sum of the
	fraction of all-pairs shortest paths that pass through $v$

	.. math::

	c_B(v) =\sum_{s,t \in V} \frac{\sigma(s, t|v)}{\sigma(s, t)}

	where $V$ is the set of nodes, $\sigma(s, t)$ is the number of
	shortest $(s, t)$-paths, and $\sigma(s, t|v)$ is the number of
	those paths passing through some node $v$ other than $s, t$.
	If $s = t$, $\sigma(s, t) = 1$, and if $v \in {s, t}$,
	$\sigma(s, t|v) = 0$ [2]_.

	Parameters
	----------
	G : graph
	A NetworkX graph.

	k : int, optional (default=None)
	If k is not None use k node samples to estimate betweenness.
	The value of k <= n where n is the number of nodes in the graph.
	Higher values give better approximation.

	normalized : bool, optional
	If True the betweenness values are normalized by `2/((n-1)(n-2))`
	for graphs, and `1/((n-1)(n-2))` for directed graphs where `n`
	is the number of nodes in G.

	weight : None or string, optional (default=None)
	If None, all edge weights are considered equal.
	Otherwise holds the name of the edge attribute used as weight.

	endpoints : bool, optional
	If True include the endpoints in the shortest path counts.

	Returns
	-------
	nodes : dictionary
	Dictionary of nodes with betweenness centrality as the value.

	
	Notes
	-----
	The algorithm is from Ulrik Brandes [1]_.
	See [4]_ for the original first published version and [2]_ for details on
	algorithms for variations and related metrics.

	For approximate betweenness calculations set k=#samples to use
	k nodes ("pivots") to estimate the betweenness values. For an estimate
	of the number of pivots needed see [3]_.

	For weighted graphs the edge weights must be greater than zero.
	Zero edge weights can produce an infinite number of equal length
	paths between pairs of nodes.

	
	"""
	betweenness = dict.fromkeys(G, 0.0) # b[v]=0 for v in G
	if k is None:
		nodes = G
	else:
		random.seed(seed)
		nodes = random.sample(G.nodes(), k)
	for s in nodes:

		# single source shortest paths
		if weight is None: # use BFS
			S, P, sigma = _single_source_shortest_path_basic(G, s)
		else: # use Dijkstra's algorithm
			S, P, sigma = _single_source_dijkstra_path_basic(G, s, weight)

		# accumulation
		if endpoints:
			betweenness = _accumulate_endpoints(betweenness, S, P, sigma, s)
		else:
			betweenness = _accumulate_basic(betweenness, S, P, sigma, s)

	# rescaling
	betweenness = _rescale(betweenness, len(G), normalized=normalized,
						directed=G.is_directed(), k=k)
	return betweenness


In [3]:
import networkx as nx
g=nx.erdos_renyi_graph(50,0.5)
b=nx.betweenness_centrality(g)
print(b)

{0: 0.01077071215876738, 1: 0.01279537178396722, 2: 0.010238366561520822, 3: 0.008702083966564758, 4: 0.017912393252397674, 5: 0.013155027366205262, 6: 0.013304707964940479, 7: 0.0064817921285508315, 8: 0.0076240978270540076, 9: 0.014298916300341871, 10: 0.010796289512644473, 11: 0.009940201383028514, 12: 0.008769812764272392, 13: 0.01067112811935741, 14: 0.018471577201812715, 15: 0.011644070349921112, 16: 0.007394694803276656, 17: 0.00872773731654126, 18: 0.015783852511537005, 19: 0.012925086854033433, 20: 0.012366213602826668, 21: 0.007569292961443244, 22: 0.010470074139160193, 23: 0.010109610620308324, 24: 0.008163427325203879, 25: 0.010199704914827808, 26: 0.012671587792129428, 27: 0.018903990306301233, 28: 0.006004575628675267, 29: 0.006246065414732881, 30: 0.022789071279585908, 31: 0.012134481646529884, 32: 0.0137093299369394, 33: 0.01030523533761671, 34: 0.008838055562995537, 35: 0.005701158025706265, 36: 0.009781210376316837, 37: 0.010090403386965431, 38: 0.0067162109971668, 39